In [23]:
import fingerprint_feature_extractor
import fingerprint_enhancer
import cv2
import math
import heapq

from Minutia.Minutia import Minutia
from Minutia.fingerprint_feature_extractor import FingerprintFeatureExtractor

!pip install fingerprint_feature_extractor
!pip install fingerprint_enhancer


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [25]:
# equation 10
def ComputeGlobalFeatureVectors(minutiaeList, referenceMinutiae):
    for index, minutia in enumerate(minutiaeList):
        tempFeatureVector = {}
        tempFeatureVector["r_kb"] = euclideanDistance(referenceMinutiae.locX, referenceMinutiae.locY, \
            minutia.locX, minutia.locY)
        tempFeatureVector["theta_kb"] = radialAngle(minutia.locX, minutia.locY, referenceMinutiae.orientation, \
            referenceMinutiae.locX, referenceMinutiae.locY)
        tempFeatureVector["phi_kb"] = minutiaDirection(minutia.orientation, referenceMinutiae.orientation)

        minutiaeList[index].globalFeatureVector = tempFeatureVector

In [26]:
# equation 6
def ComputeFeatureVectors(minutiaeList):
    for index, minutia in enumerate(minutiaeList):
        tempFeatureVector = {}
        # Compute euclidean distance for nearest neighbor (neighbor-i) and second nearest neighbor (neighbor-j)
        tempFeatureVector["d_ki"] = euclideanDistance(minutia.locX, minutia.locY,
            minutiaeList[minutia.closestNeighbor].locX, minutiaeList[minutia.closestNeighbor].locY)
        tempFeatureVector["d_kj"] = euclideanDistance(minutia.locX, minutia.locY,
            minutiaeList[minutia.secondClosestNeighbor].locX, minutiaeList[minutia.secondClosestNeighbor].locY)
        # Compute relative radial angle for nearest neighbor (neighbor-i) and second nearest neighbor (neighbor-j)
        tempFeatureVector["theta_ki"] = radialAngle(minutia.locX, minutia.locY, minutia.orientation,
            minutiaeList[minutia.closestNeighbor].locX, minutiaeList[minutia.closestNeighbor].locY)
        tempFeatureVector["theta_kj"] = radialAngle(minutia.locX, minutia.locY, minutia.orientation,
            minutiaeList[minutia.secondClosestNeighbor].locX, minutiaeList[minutia.secondClosestNeighbor].locY)
        # Compute relative direction for nearest neighbor (neighbor-i) and second nearest neighbor (neighbor-j)
        tempFeatureVector["phi_ki"] = minutiaDirection(minutia.orientation, minutiaeList[minutia.closestNeighbor].orientation)
        tempFeatureVector["phi_kj"] = minutiaDirection(minutia.orientation, minutiaeList[minutia.secondClosestNeighbor].orientation)
        # Determine the minutia type for nearest neighbor (neighbor-i) and second nearest neighbor (neighbor-j)
        tempFeatureVector["t_k"] = minutia.minutiaType
        tempFeatureVector["t_i"] = minutiaeList[minutia.closestNeighbor].minutiaType
        tempFeatureVector["t_j"] = minutiaeList[minutia.secondClosestNeighbor].minutiaType

        minutiaeList[index].featureVector = tempFeatureVector

In [27]:
# Find the TWO nearest minutia neighbors for each given minutia
def findMinutiaNeighbors(minutiaeList):
    # needleMinutia is the minutia for which we're searching its closest neighbors
    # needleIndex is its index in the minutiaeList
    for needleIndex, needleMinutia in enumerate(minutiaeList):
        # here we go over all the minutia in the minutiaeList and we calculate the euclidean
        # distance between it and our needle minutia and store the results in a dictionary
        minutiaIndexAndDistance = {}
        for index, minutia in enumerate(minutiaeList):
            distance = euclideanDistance(minutia.locX, minutia.locY, needleMinutia.locX, needleMinutia.locY)
            # if the euclidean distance is equal to 0, that means that the needleMinutia and minutia are
            # the same object. Therefore we don't add its distance to the list
            if distance != 0:
                minutiaIndexAndDistance[index] = distance

        # now we take the two closest neighbors to our needleMinutia based on the values that
        # are stored in our minutiaIndexAndDistance dictionary
        twoClosestNeighbors = heapq.nsmallest(2, minutiaIndexAndDistance.items(), key=lambda x: x[1])
        minutiaeList[needleIndex].closestNeighbor = twoClosestNeighbors[0][0]
        minutiaeList[needleIndex].secondClosestNeighbor = twoClosestNeighbors[1][0]

In [28]:
# Calculate the difference between two angles in radians
def angleDifference(t1, t2):
    if t1 - t2 > -math.pi and t1-t2 <= math.pi:
        return t1-t2
    elif t1 - t2 <= -math.pi:
        return 2 * math.pi + t1 - t2
    else:
        return 2 * math.pi - t1 + t2

In [29]:
# equation 9
# find a pair of minutiae with the highest similarity
def findBestMatch(inputMinutiaeList, templateMinutiaeList):
    bestMatch = {}
    maxSimilarity = -1

    for inputMinutia in inputMinutiaeList:
        for templateMinutia in templateMinutiaeList:
            similarity = compareLocalStructures(inputMinutia.featureVector, templateMinutia.featureVector)
            if (similarity > maxSimilarity):
                bestMatch["inputMinutia"] = inputMinutia
                bestMatch["templateMinutia"] = templateMinutia
                bestMatch["similarity"] = similarity
                maxSimilarity = similarity
    return bestMatch

In [41]:
# equation 3
# Calculate the Euclidean distance between point1 (x1, y1) and point2 (x2, y2)
def euclideanDistance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [31]:
# equation 4
# calculate the relative radial angle
def radialAngle(x1, y1, orientation1 ,x2, y2):
    if abs(x1 - x2) != 0:
        return angleDifference(math.atan((y1 - y2)/(x1 - x2)), orientation1)
    else:
        if (y1 - y2 >= 0):
            return angleDifference(math.pi/2, orientation1)
        else:
            return angleDifference(-math.pi/2, orientation1)

In [42]:
# equation 5
# calculate the relative direction
def minutiaDirection(orientation1, orientation2):
    return angleDifference(orientation1, orientation2)

In [43]:
def computeVectorMagnitude(vector):
    sum = 0
    for key, value in vector.items():
        sum += value**2
    return math.sqrt(sum)

In [48]:
# equation 7
# compares a minutiae pair from the inputImage and templateImage. A value of 1 indicates a full match and a value of 0 indicates no match
# porównuje parę minucji z inputImage i templateImage. Wartość 1 oznacza pełne dopasowanie, a wartość 0 oznacza brak dopasowania
def compareLocalStructures(inputFeatureVector, templateFeatureVector):
    weights = {"d_ki":1, "d_kj":1, "theta_ki":0.3*180/math.pi, "theta_kj":0.3*180/math.pi, \
        "phi_ki":0.3*180/math.pi, "phi_kj":0.3*180/math.pi, "t_k":3, "t_i": 3, "t_j":3}
    m = len(weights)
    print('m', m)
    bl = 6 * m
    print('bl', bl)
    vector = {}
    features = ["d_ki", "d_kj", "t_k", "t_i", "t_j"]
    for feature in features:
        vector[feature] = (inputFeatureVector[feature] - templateFeatureVector[feature]) * weights[feature]
        print('vector[feature]', vector[feature])
    angles = ["theta_ki", "theta_kj", "phi_ki", "phi_kj"]
    for angle in angles:
        vector[angle] = angleDifference(inputFeatureVector[angle], templateFeatureVector[angle]) * weights[angle]
        print('vector[angle]', vector[angle])
    vectorMagnitude = computeVectorMagnitude(vector)
    print('vectorMagnitude', vectorMagnitude)
    if bl > vectorMagnitude:
        print('wynik',(bl - vectorMagnitude) / bl)
        return (bl - vectorMagnitude) / bl
    else:
        return 0

# Local

Dopasowywanie odcisków palców na podstawie
struktur lokalnych i globalnych minucji

In [45]:
def local_m(base, compare):
  # Load the fingerprint images
  inputImg = cv2.imread(f"./fingerprints/{compare}.tif", 0)
  templateImg = cv2.imread(f"./fingerprints/{base}.tif", 0)
  #inputImg = cv2.imread("./fingerprints/1.tif", 0)
  #templateImg = cv2.imread("./fingerprints/2.tif", 0)


  # Enhance the images (may throw an exception for low dpi images)
  # Popraw obrazy (może zgłosić wyjątek dla obrazów o niskiej rozdzielczości)
  enInputImg = fingerprint_enhancer.enhance_Fingerprint(inputImg)
  enTemplateImg = fingerprint_enhancer.enhance_Fingerprint(templateImg)

  # Extract the features from the fingerprint image
  # Wyodrębnij funkcje z obrazu odcisku palca
  inputFeaturesTerminations, inputFeaturesBifurcations = fingerprint_feature_extractor.extract_minutiae_features(enInputImg, spuriousMinutiaeThresh=20, invertImage=False, showResult=False, saveResult=True)
  inputRes = cv2.imread("./result.png", cv2.IMREAD_COLOR)
  templateFeaturesTerminations, templateFeaturesBifurcations = fingerprint_feature_extractor.extract_minutiae_features(enTemplateImg, spuriousMinutiaeThresh=20, invertImage=False, showResult=False, saveResult=True)
  templateRes = cv2.imread("./result.png", cv2.IMREAD_COLOR)

  # Create a list of minutiae features (x location, y location, orientation in degrees, type)
  # Utwórz listę minucji (lokalizacja x, lokalizacja y, orientacja w stopniach, typ)
  inputMinutiaeList = []
  templateMinutiaeList = []
  for term in inputFeaturesTerminations:
      inputMinutiaeList.append(Minutia(term.locX, term.locY, math.radians(term.Orientation[0]), 0))
  for bif in inputFeaturesBifurcations:
      inputMinutiaeList.append(Minutia(bif.locX, bif.locY, math.radians(bif.Orientation[0]), 1))

  for term in templateFeaturesTerminations:
      templateMinutiaeList.append(Minutia(term.locX, term.locY, math.radians(term.Orientation[0]), 0))
  for bif in templateFeaturesBifurcations:
      templateMinutiaeList.append(Minutia(bif.locX, bif.locY, math.radians(bif.Orientation[0]), 1))

  #Znajdź dwóch najbliższych sąsiadów dla każdej minucji
  # Find two nearest neighbors for each minutia
  findMinutiaNeighbors(inputMinutiaeList)
  findMinutiaNeighbors(templateMinutiaeList)

  # Oblicz wektor cech dla każdej minucji
  ComputeFeatureVectors(inputMinutiaeList)
  ComputeFeatureVectors(templateMinutiaeList)

  i = 0
  sum = 0
  am = min(len(inputMinutiaeList), len(templateMinutiaeList))-3
  while i < am:
      bestMatch = findBestMatch(inputMinutiaeList, templateMinutiaeList)
      # print(bestMatch["inputMinutia"])
      # print(bestMatch["templateMinutia"])
      inputMinutiaeList.remove(bestMatch['inputMinutia'])
      templateMinutiaeList.remove(bestMatch['templateMinutia'])
      sum += bestMatch['similarity']
      i += 1
      # Find two nearest neighbors for each minutia
      findMinutiaNeighbors(inputMinutiaeList)
      findMinutiaNeighbors(templateMinutiaeList)

      # Compute the feature vector for each minutia
      ComputeFeatureVectors(inputMinutiaeList)
      ComputeFeatureVectors(templateMinutiaeList)

  # print("Liczba dopasowań", am)
  return str(sum/am)
  # cv2_imshow(inputImg)
  # cv2_imshow(inputRes)
  # cv2_imshow(templateImg)
  # cv2_imshow(templateRes)

# Global

Dopasowywanie odcisków palców na podstawie
struktur lokalnych i globalnych minucji

In [46]:

# equation 9, but for one list and one minutia
def findBestMatchOne(inputMinutiaeList, templateMinutia):
    bestMatch = -1
    maxSimilarity = -1
    result = 0

    for inputMinutia in inputMinutiaeList:
        bestMatch = bestMatch +1
        similarity = compareLocalStructures(inputMinutia.featureVector, templateMinutia.featureVector)
        if (similarity > maxSimilarity):
            maxSimilarity = similarity
            result = bestMatch
    return result

# equation 11
def calculateMatchingCertainty(inputMinutia, templateMinutia):
    boundingBox = {"r": 64, "theta": math.pi/6, "phi": math.pi/6}
    boundingBoxMagnitude = computeVectorMagnitude(boundingBox)

    inputFeatureVector = inputMinutia.featureVector
    inputGlobalFeatureVector = inputMinutia.globalFeatureVector
    templateFeatureVector = templateMinutia.featureVector
    templateGlobalFeatureVector = templateMinutia.globalFeatureVector

    globalFeatureVector = {}
    globalFeatureVector["r_kb"] = inputGlobalFeatureVector["r_kb"] - templateGlobalFeatureVector["r_kb"]
    globalFeatureVector["theta_kb"] = angleDifference(inputGlobalFeatureVector["theta_kb"],templateGlobalFeatureVector["theta_kb"])
    globalFeatureVector["phi_kb"]  = angleDifference(inputGlobalFeatureVector["phi_kb"],templateGlobalFeatureVector["phi_kb"])
    globalFeatureVectorMagnitude = computeVectorMagnitude(globalFeatureVector)

    if boundingBoxMagnitude > globalFeatureVectorMagnitude:
        return 0.5 + 0.5 * compareLocalStructures(inputFeatureVector, templateFeatureVector)
    else:
        return 0

# equation 12
def calculateFinalMatchingScore(inputMinutiaeList, templateMinutiaeList):
    sum = 0
    for i in range(0, len(inputMinutiaeList)):
        sum += calculateMatchingCertainty(inputMinutiaeList[i], templateMinutiaeList[findBestMatchOne(templateMinutiaeList, inputMinutiaeList[i])])
    return 1 * sum/len(inputMinutiaeList)

def global_m(base, compare):
  # Load the fingerprint images
  inputImg = cv2.imread(f"./fingerprints/{compare}.tif", 0)
  templateImg = cv2.imread(f"./fingerprints/{base}.tif", 0)

  # Enhance the images (may throw an exception for low dpi images)
  enInputImg = fingerprint_enhancer.enhance_Fingerprint(inputImg)
  enTemplateImg = fingerprint_enhancer.enhance_Fingerprint(templateImg)

  # Extract features from the fingerprint images
  inputFeaturesTerminations, inputFeaturesBifurcations = fingerprint_feature_extractor.extract_minutiae_features(enInputImg, spuriousMinutiaeThresh=20, invertImage=False, showResult=False, saveResult=True)
  inputRes = cv2.imread("./result.png", cv2.IMREAD_COLOR)
  templateFeaturesTerminations, templateFeaturesBifurcations = fingerprint_feature_extractor.extract_minutiae_features(enTemplateImg, spuriousMinutiaeThresh=20, invertImage=False, showResult=False, saveResult=True)
  tempinputRes = cv2.imread("./result.png", cv2.IMREAD_COLOR)
  
  # Create a list of minutia features (x location, y location, orientation in degrees, type)
  inputMinutiaeList = []
  templateMinutiaeList = []
  for term in inputFeaturesTerminations:
      inputMinutiaeList.append(Minutia(term.locX, term.locY, math.radians(term.Orientation[0]), 0))
  for bif in inputFeaturesBifurcations:
      inputMinutiaeList.append(Minutia(bif.locX, bif.locY, math.radians(bif.Orientation[0]), 1))

  for term in templateFeaturesTerminations:
      templateMinutiaeList.append(Minutia(term.locX, term.locY, math.radians(term.Orientation[0]), 0))
  for bif in templateFeaturesBifurcations:
      templateMinutiaeList.append(Minutia(bif.locX, bif.locY, math.radians(bif.Orientation[0]), 1))


  # Find two nearest neighbors for each minutia
  findMinutiaNeighbors(inputMinutiaeList)
  findMinutiaNeighbors(templateMinutiaeList)

  # Compute the feature vector for each minutia
  ComputeFeatureVectors(inputMinutiaeList)
  ComputeFeatureVectors(templateMinutiaeList)


  bestMatch = findBestMatch(inputMinutiaeList, templateMinutiaeList)

  # print("Best Match Details:")
  # print("Input Minutia:", bestMatch["inputMinutia"].__dict__)
  # print("Template Minutia:", bestMatch["templateMinutia"].__dict__)
  # print("Similarity:", bestMatch["similarity"])

  # Calculate featureVector relative to the minutiae from bestMatch
  ComputeGlobalFeatureVectors(inputMinutiaeList, bestMatch["inputMinutia"])
  ComputeGlobalFeatureVectors(templateMinutiaeList, bestMatch["templateMinutia"])
  # print(templateMinutiaeList)
  result = calculateFinalMatchingScore(inputMinutiaeList, templateMinutiaeList)
  return str(result)

  # Print the inputMinutiaeList details
  # print("\nInput Minutiae List:")
  #for minutia in inputMinutiaeList:
  #    print(minutia.__dict__)

  # Print the templateMinutiaeList details
  # print("\nTemplate Minutiae List:")
  #for minutia in templateMinutiaeList:
    #  for minutia in inputMinutiaeList:
  #        minutia_dict = minutia.__dict__.copy()  # Kopia słownika, aby nie modyfikować oryginalnego
          #   minutia_dict['locX'] = round(minutia_dict['locX'], 2)
          #   minutia_dict['locY'] = round(minutia_dict['locY'], 2)
      #   minutia_dict['orientation'] = round(minutia_dict['orientation'], 2)
          #print(minutia_dict)
      #print(minutia.__dict__)

  # cv2_imshow(inputImg)
  # cv2_imshow(inputRes)
  # cv2_imshow(templateImg)
  # cv2_imshow(templateRes)

In [49]:
import csv

f = open('csv_data.csv', 'w+')
writer = csv.writer(f, delimiter=';')

for k in range(1):
  print(f"TABLE {k+1}")
  writer.writerow([f"TABLE {k+1}"] * 10)
  for i in range(1):
    print(f"ROW {i+1}")
    base = f"finger_{i+1}/10{i+1}_1"
    row = []

    for j in range(1):
      compare = f"finger_{j+1}/10{j+1}_{k+1}"
      row.append(local_m(base, compare))
      row.append(global_m(base, compare))

    writer.writerow(row)

f.close()

TABLE 1
ROW 1
m 9
bl 54
vector[feature] 0.0
vector[feature] 0.0
vector[feature] 0
vector[feature] 0
vector[feature] 0
vector[angle] 0.0
vector[angle] 0.0
vector[angle] 0.0
vector[angle] 0.0
vectorMagnitude 0.0
wynik 1.0
m 9
bl 54
vector[feature] 22.787877919056452
vector[feature] 23.097415177239654
vector[feature] 0
vector[feature] 0
vector[feature] 0
vector[angle] 40.63255830148447
vector[angle] 51.633495166533635
vector[angle] -46.0304846468766
vector[angle] 7.969515353123397
vectorMagnitude 86.90293019771887
m 9
bl 54
vector[feature] 12.205782733714411
vector[feature] 24.482647605340937
vector[feature] 0
vector[feature] 0
vector[feature] 0
vector[angle] 28.248418707449975
vector[angle] 24.732279905687633
vector[angle] 0.0
vector[angle] -5.530484646876602
vectorMagnitude 46.782755959263184
wynik 0.13365266742105214
m 9
bl 54
vector[feature] 13.600743995507653
vector[feature] 34.493467267759314
vector[feature] 0
vector[feature] 0
vector[feature] 0
vector[angle] 5.315636663235476
vecto